# **Task 6**


Develop a chatbot that can answer frequently asked questions (FAQs) about a company, such as:

1. What products/services do you offer?
2. What are your business hours?
3. How can I contact customer support?

Requirements
1. Define intents and responses
2. Train a machine learning model to recognize user queries
3. Integrate with a chatbot platform

Tools
You can use libraries like Dialogflow, NLTK, or spaCy.

In [7]:
!pip install nltk tensorflow

# Import libraries
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

import json
import numpy as np
import random
import pickle
import re

from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD

lemmatizer = WordNetLemmatizer()

# Define intents manually
data = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["Hi", "Hello", "How are you?", "Is anyone there?"],
            "responses": ["Hello!", "Hi there!", "Greetings!"]
        },
        {
            "tag": "products",
            "patterns": ["What products do you offer?", "Tell me about your services"],
            "responses": ["We offer consulting, software development, and support."]
        },
        {
            "tag": "hours",
            "patterns": ["What are your hours?", "When are you open?"],
            "responses": ["Our business hours are 9am to 5pm, Monday to Friday."]
        },
        {
            "tag": "support",
            "patterns": ["How do I contact support?", "Help me!", "I need assistance"],
            "responses": ["You can contact our support team at support@example.com."]
        }
    ]
}

# Preprocessing
words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

# Use regex tokenizer instead of nltk.word_tokenize
def simple_tokenize(sentence):
    return re.findall(r'\b\w+\b', sentence.lower())

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        word_list = simple_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent["tag"]))
        if intent["tag"] not in classes:
            classes.append(intent["tag"])

words = [lemmatizer.lemmatize(w) for w in words if w not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))

# Training data
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    pattern_words = [lemmatizer.lemmatize(w) for w in document[0]]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

# Build model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Train
model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)

# Save model and data
model.save("chatbot_model.h5")
pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.2614 - loss: 1.3433
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.1409 - loss: 1.4645    
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0705 - loss: 1.5692    
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4773 - loss: 1.3356
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.3068 - loss: 1.3002
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5727 - loss: 1.1894
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3818 - loss: 1.2928
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7636 - loss: 1.1812
Epoch 9/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6886 - loss: 1.1755
Epoch 10/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7591 - loss: 1.1701
Epoch 11/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6182 - loss: 1.0070
Epoch 12/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8

In [9]:
from tensorflow.keras.models import load_model
import random

model = load_model("chatbot_model.h5")
words = pickle.load(open("words.pkl", "rb"))
classes = pickle.load(open("classes.pkl", "rb"))

def clean_up(sentence):
    sentence_words = re.findall(r'\b\w+\b', sentence.lower())
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence, words):
    sentence_words = clean_up(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence, words)
    res = model.predict(np.array([bow]))[0]
    threshold = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > threshold]
    results.sort(key=lambda x: x[1], reverse=True)
    return [{"intent": classes[r[0]], "probability": str(r[1])} for r in results]

def get_response(intents_list):
    tag = intents_list[0]['intent']
    for intent in data["intents"]:
        if intent["tag"] == tag:
            return random.choice(intent["responses"])

# Try chatting
while True:
    message = input("You: ")
    if message.lower() == "quit":
        break
    intents_list = predict_class(message)
    result = get_response(intents_list)
    print("Bot:", result)


You: Hi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
Bot: Hello!
You: What products do you offer?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Bot: We offer consulting, software development, and support.
You: How can I contact support?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Bot: You can contact our support team at support@example.com.
You: What are your hours?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Bot: Our business hours are 9am to 5pm, Monday to Friday.
You: ok
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Bot: Hello!
You: quit
